In [ ]:
import os
import boto3
import pandas as pd
from IPython.display import display
from io import BytesIO
import matplotlib.pyplot as plt
from pathlib import Path
import json
import seaborn as sns

# **Proyecto Final - Pasantía de Ingeniería de Datos**

Para generar las credenciales ejecuto este comando en terminal: 

In [ ]:
# aws sts get-session-token --duration-seconds 14400 --output json | Out-File -FilePath "$env:USERPROFILE\aws-temp-creds.json" -Encoding utf8

In [2]:
# credenciales
path = Path.home() / "aws-temp-creds.json"

with open(path, "r", encoding="utf-8-sig") as f:
    creds = json.load(f)["Credentials"]

os.environ["AWS_ACCESS_KEY_ID"] = creds["AccessKeyId"]
os.environ["AWS_SECRET_ACCESS_KEY"] = creds["SecretAccessKey"]
os.environ["AWS_SESSION_TOKEN"] = creds["SessionToken"]
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

sts = boto3.client("sts")
print(sts.get_caller_identity())

{'UserId': 'AIDAVNFKRAMYBQN25WWQO', 'Account': '371872301872', 'Arn': 'arn:aws:iam::371872301872:user/ezequiel.coggiola', 'ResponseMetadata': {'RequestId': 'f50d7f03-b0ea-4a2f-b8af-060752216009', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f50d7f03-b0ea-4a2f-b8af-060752216009', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6UzoxNzYzNjQ2NDc2MzM0OlI6VndGUExzZFk=', 'content-type': 'text/xml', 'content-length': '414', 'date': 'Thu, 20 Nov 2025 13:47:56 GMT'}, 'RetryAttempts': 0}}


In [5]:
# conexión a s3

region = os.environ.get("AWS_REGION", "us-east-1")
dataset_name = "ecommerce"
bucket_name = f"{dataset_name}-ezequiel-2025"
folders = ["raw/", "processed/", "curated/"]

boto_sess = boto3.Session(region_name=region)

s3 = boto_sess.client("s3")

In [5]:
# 1. Crear el bucket (en us-east-1 no se usa LocationConstraint)
s3.create_bucket(Bucket=bucket_name)
print(f"✅ Bucket '{bucket_name}' creado en {region}")

# 2. Activar versioning
s3.put_bucket_versioning(
    Bucket=bucket_name,
    VersioningConfiguration={"Status": "Enabled"}
)

# 3. Configurar default encryption (SSE-S3)
s3.put_bucket_encryption(
    Bucket=bucket_name,
    ServerSideEncryptionConfiguration={
        "Rules": [
            {
                "ApplyServerSideEncryptionByDefault": {
                    "SSEAlgorithm": "AES256"  # SSE-S3
                }
            }
        ]
    }
)

# 4. Block Public Access
s3.put_public_access_block(
    Bucket=bucket_name,
    PublicAccessBlockConfiguration={
        "BlockPublicAcls": True,
        "IgnorePublicAcls": True,
        "BlockPublicPolicy": True,
        "RestrictPublicBuckets": True
    }
)

# 5. Agregar tags al bucket
s3.put_bucket_tagging(
    Bucket=bucket_name,
    Tagging={
        "TagSet": [
            {"Key": "owner", "Value": "Ezequiel Coggiola"},
            {"Key": "team", "Value": "Datera"},
            {"Key": "env", "Value": "dev"},
            {"Key": "dataset", "Value": dataset_name}
        ]
    }
)

# 6. Crear la estructura de "carpetas" (raw/, processed/, curated/)
for folder in folders:
    s3.put_object(Bucket=bucket_name, Key=folder)
    print(f"   - Creada estructura: {folder}")

✅ Bucket 'ecommerce-ezequiel-2025' creado en us-east-1
   - Creada estructura: raw/
   - Creada estructura: processed/
   - Creada estructura: curated/


In [9]:
import os
import boto3

s3 = boto3.client("s3")
dataset_name = "ecommerce_ezequiel"   # o pasalo como parámetro

def upload_files(local_folder, bucket_name, layer="raw", file_types=None):
    """
    Sube archivos a S3 creando automáticamente una carpeta por dataset.
    Ejemplo:
        raw/customers/customers.csv
        raw/orders/orders.csv
    """
    if file_types is None:
        file_types = ["csv", "parquet"]

    print(f"📂 Procesando carga para capa: '{layer}'...")

    for filename in os.listdir(local_folder):

        local_path = os.path.join(local_folder, filename)

        # Validación: que sea archivo y que tenga extensión válida
        if not os.path.isfile(local_path):
            continue
            
        ext = filename.split(".")[-1].lower()
        
        if ext in file_types:
            # Nombre del dataset = filename sin extensión
            dataset_folder = filename.split(".")[0]

            # S3 path: raw/customers/customers.csv
            s3_key = f"{layer}/{dataset_folder}/{filename}"

            # Tags
            tags = (
                f"layer={layer}&owner=Ezequiel Coggiola&team=Datera"
                f"&env=dev&dataset={dataset_name}"
            )

            try:
                s3.upload_file(
                    local_path,
                    bucket_name,
                    s3_key,
                    ExtraArgs={"Tagging": tags}
                )
                print(f"   ✅ Subido: {filename} -> {s3_key}")
            except Exception as e:
                print(f"   ❌ Error subiendo {filename}: {str(e)}")

# Ejemplo de uso:
# upload_files("./mis_datos", bucket_name, layer="raw")


In [ ]:
# Ejecutar

local_folder = "../dataset-ecommerce"

upload_files(local_folder, bucket_name, layer="raw", file_types="csv")

response = s3.list_objects_v2(Bucket=bucket_name, Prefix="raw/2025/")
for obj in response.get("Contents", []):
    print(obj["Key"])

📂 Procesando carga para capa: 'raw'...
   ✅ Subido: customers.csv -> raw/customers/customers.csv
   ✅ Subido: events.csv -> raw/events/events.csv
   ✅ Subido: orders.csv -> raw/orders/orders.csv
   ✅ Subido: order_items.csv -> raw/order_items/order_items.csv
   ✅ Subido: products.csv -> raw/products/products.csv
   ✅ Subido: reviews.csv -> raw/reviews/reviews.csv
   ✅ Subido: sessions.csv -> raw/sessions/sessions.csv


: 

**Correr en terminal para ejecutar .sql en RDS:**

    -- password: benji1234

    -- Me conecto a la instancia, y ejecuto el archivo

    mysql --local-infile=1 -h "ENDPOINT" -u admin -p < /home/ezecoggiola/proyectos/rds/load_files_rds.sql


